In [2]:
import cv2
import os

data_path='Training'
categories=os.listdir(data_path)
categories.remove('.DS_Store')
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

{'Man': 0, 'Woman': 1}
['Man', 'Woman']
[0, 1]


In [3]:
img_size=32
data=[]
target=[]

facedata = "haarcascade_frontalface_default.xml"
cascade = cv2.CascadeClassifier(facedata)


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)

        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        faces = cascade.detectMultiScale(img)
        try:
            for f in faces:
                x, y, w, h = [v for v in f]
                sub_face = img[y:y + h, x:x + w]
                gray=cv2.cvtColor(sub_face,cv2.COLOR_BGR2GRAY)           
                resized=cv2.resize(gray,(img_size,img_size))
                data.append(resized)
                target.append(label_dict[category])
        except Exception as e:
            print('Exception:',e)

In [4]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data=np.array(data)/255.0

data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

np.save('./training/data',data)
np.save('./training/target',new_target)




In [5]:
import numpy as np

data=np.load('./training/data.npy')
target=np.load('./training/target.npy')

In [6]:
data.shape[1:]

(32, 32, 1)

In [7]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint


noOfFilters=64
sizeOfFilter1=(3,3)
sizeOfFilter2=(3,3)
sizeOfPool=(2,2)
noOfNode=64



model=Sequential()
model.add((Conv2D(64, sizeOfFilter1, input_shape=data.shape[1:],activation='relu')))
model.add((Conv2D(64, sizeOfFilter1,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add((Conv2D(128, sizeOfFilter2,activation='relu')))
model.add((Conv2D(128, sizeOfFilter2,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(noOfNode, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

2023-01-19 22:50:08.566582: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [9]:
checkpoint = ModelCheckpoint('./training/model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=50,callbacks=[checkpoint],validation_split = 0.2)

Epoch 1/50
366/367 [============================>.] - ETA: 0s - loss: 0.5415 - accuracy: 0.6894

2023-01-19 22:50:57.107116: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./training/model-001.model/assets
367/367 [==============================] - 36s 96ms/step - loss: 0.5419 - accuracy: 0.6893 - val_loss: 0.2338 - val_accuracy: 0.9130
Epoch 2/50
367/367 [==============================] - 36s 98ms/step - loss: 0.2431 - accuracy: 0.9095 - val_loss: 0.1697 - val_accuracy: 0.9386
Epoch 3/50
367/367 [==============================] - 35s 96ms/step - loss: 0.1871 - accuracy: 0.9325 - val_loss: 0.1590 - val_accuracy: 0.9407
Epoch 4/50
367/367 [==============================] - 35s 94ms/step - loss: 0.1665 - accuracy: 0.9411 - val_loss: 0.1850 - val_accuracy: 0.9246
Epoch 5/50
367/367 [==============================] - 35s 96ms/step - loss: 0.1501 - accuracy: 0.9475 - val_loss: 0.1486 - val_accuracy: 0.9512
Epoch 6/50
367/367 [==============================] - 36s 97ms/step - loss: 0.1404 - accuracy: 0.9493 - val_loss: 0.1465 - val_accuracy: 0.9485
Epoch 7/50
367/367 [==============================] - 36s 97ms/step - loss: 0.